In [21]:
# Useful starting lines
import numpy as np
from proj1_helpers import *
%load_ext autoreload
%autoreload 2
%matplotlib inline

# from costs import compute_mse, compute_loss
# from plots import *
from helpers import *
# from grid_search import *
# import datetime
# from ridge_regression import *
# from gradient_descent import *
# from stochastic_gradient_descent import *
from implementations import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
DATA_PATH_TRAIN = '../data/train.csv'
# y_train, data_train, ids_tr = load_csv_data(DATA_PATH_TRAIN, sub_sample=True)
y_train, data_train, ids_tr = load_csv_data(DATA_PATH_TRAIN, sub_sample=False)

In [3]:
DATA_PATH_TEST = '../data/test.csv'
_, data_test, ids_te = load_csv_data(DATA_PATH_TEST, sub_sample=True)

In [4]:
print(data_train.shape)
print(data_test.shape)

(250000, 30)
(11365, 30)


### Features pre-processing

In [38]:
data_train= put_NaN(data_train)
datasets_train = divide_subset(data_train, y_train)

C:\Users\prisgdd\Documents\EPFL\MachineLearning\projects\ML_project1\scripts\helpers.py:210: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  def divide_subset(data, y = None):


In [39]:
data_test = put_NaN(data_test)
datasets_test = divide_subset(data_test)

In [40]:
test = True
# test = False

# Preprocess datasets_train
for ind, subset_train in enumerate(datasets_train):
    print('Train, before preprocessing : {} '.format(subset_train[0].shape))
    if ind == 0:
        subset_train[0] = preprocess_data_train(subset_train[0], jet = 0)
    elif ind == 1:
        subset_train[0] = preprocess_data_train(subset_train[0], jet = 1)
    elif ind == 2:
        subset_train[0] = preprocess_data_train(subset_train[0], jet = 23)

    print('Train, after preprocessing : {} '.format(subset_train[0].shape))
    datasets_train[ind][0] = subset_train[0]
if test == True:
    print("")
    # Preprocess datasets_train
    for ind, subset_test in enumerate(datasets_test):
        subset_train = datasets_train[ind][0]

        print('Test, before preprocessing : {} '.format(subset_test[0].shape))

        if ind == 0:
            subset_test[0] = preprocess_data_test(subset_test[0], subset_train, jet = 0)
        elif ind == 1:
            subset_test[0] = preprocess_data_test(subset_test[0], subset_train, jet = 1)
        elif ind == 2:
            subset_test[0] = preprocess_data_test(subset_test[0], subset_train, jet = 23)

        print('Test, after preprocessing : {} '.format(subset_test[0].shape))
        datasets_test[ind][0] = subset_test[0]

Train, before preprocessing : (99913, 30) 
Train, after preprocessing : (99913, 18) 
Train, before preprocessing : (77544, 30) 
Train, after preprocessing : (77544, 21) 
Train, before preprocessing : (72543, 30) 
Train, after preprocessing : (72543, 30) 

Test, before preprocessing : (4496, 30) 
Test, after preprocessing : (4496, 18) 
Test, before preprocessing : (3525, 30) 
Test, after preprocessing : (3525, 21) 
Test, before preprocessing : (3344, 30) 
Test, after preprocessing : (3344, 30) 


# 1. Least Square Gradient Descent 


In [50]:
def cross_validation(y, x, k_indices, k, method, max_iters = 0, gamma = 0, lambda_ = 0):
    """return the loss of ridge regression."""

    # get k'th subgroup in test, others in train
    ind_train = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    ind_test = k_indices[k]
    ind_train = ind_train.reshape(-1)
    
    x_train = x[ind_train]
    y_train = y[ind_train]
    x_test = x[ind_test]
    y_test = y[ind_test]
    
    # form data  -- TODO: Change for augmented features
    _, poly_train = build_model_data(x_train, y_train)
    _, poly_test = build_model_data(x_test, y_test)

    if method == least_squares_GD or method == least_squares_SGD or method == logistic_regression:
        initial_w = np.zeros([poly_train.shape[1]])
        w, loss = method(y = y_train, tx = poly_train, initial_w = initial_w, max_iters = max_iters, gamma = gamma)
        
    elif method == least_squares:
        w, loss = method(y = y_train, tx = poly_train)
        
    elif method == ridge_regression:
        w, loss = method(y = y_train, tx = poly_train, lambda_ = lambda_)
        
    elif method == reg_logistic_regression:
        initial_w = np.zeros([poly_train.shape[1]])
        w, loss = method(y = y_train, tx = poly_train, lambda_ = lambda_, initial_w = initial_w, max_iters = max_iters, gamma = gamma)
    
    # Compute prediction for train and test
    y_pred_train = predict_labels(w, poly_train)
    y_pred_test = predict_labels(w, poly_test)
    
#     print(" --- y = -1 :: Reel: {} :: Prediction: {}".format( (y_train == -1).sum(), (y_pred_train == -1).sum() )) 
#     print(" --- y = +1 :: Reel: {} :: Prediction: {}".format( (y_train == 1).sum(), (y_pred_train == 1).sum() )) 
    
    # Compute accuracy for train and test 
    accuracy_train = compute_accuracy(y_pred_train, y_train)
    accuracy_test = compute_accuracy(y_pred_test, y_test)
    
    return accuracy_train, accuracy_test, w

In [51]:
# from plots import cross_validation_visualization

# Model parameters
method = least_squares_GD

max_iters = 100
seed = 1
degree = 7
k_fold = 8
# lambdas = np.logspace(-4, 0, 30)
lambdas = []
lambdas.append(1)
gammas = np.arange(0.4, 0.8, 0.1)

y_pred_final = np.zeros((len(ids_te),1))

# For each case (jet = 0, jet = 1, jet = 2,3)
for ind, subset_train in enumerate(datasets_train[0:1]):
    print("\n\n For case jet = {} \n".format(ind))
    x = subset_train[0]
    y = subset_train[1]
    
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)

    # define lists to store the accuracy of training data and test data
    accuracy_train = []
    accuracy_test = []

    for lambda_ in lambdas:
        for gamma in gammas:
            print("\nlambda_ = {}".format(lambda_))
            print("gamma = {} \n".format(gamma))
        
            param_method = dict(max_iters = max_iters, gamma = gamma, lambda_ = lambda_)

            accuracy_train_tmp = []
            accuracy_test_tmp = []
            w_tmp = []
            # cross validation  
            for k in range(k_fold):
                accuracy_train, accuracy_test, w = cross_validation(y, x, k_indices, k, method = method, **param_method)
                accuracy_train_tmp.append(accuracy_train)
                accuracy_test_tmp.append(accuracy_test)
                w_tmp.append(w)
                print("%d - Training accuracy: %f / Test accuracy : %f" % (k, accuracy_train, accuracy_test))

            accuracy_train = np.mean(accuracy_train_tmp)
            accuracy_test = np.mean(accuracy_test_tmp)

            print("\nAverage test accuracy: {}".format(np.mean(accuracy_test)))
            print("Variance test accuracy: {}".format(np.std(accuracy_test)))
            print("Min test accuracy: {}".format(np.min(accuracy_test)))
            print("Max test accuracy: {}\n".format(np.max(accuracy_test)))

    
    
    # ****************************************
    # ****** Predict for datasets_test *******
    # ****************************************
    if test == True:
        subset_test = datasets_test[ind][0]
        _, poly_test = build_model_data(subset_test, datasets_test[ind][1])

        y_pred_crt = predict_labels(w_tmp[0], poly_test)   # TODO : for now takes the first w
        np.put(y_pred_final, datasets_test[ind][1], y_pred_crt)




 For case jet = 0 


lambda_ = 1
gamma = 0.4 

0 - Training accuracy: 0.817050 / Test accuracy : 0.817119
1 - Training accuracy: 0.817508 / Test accuracy : 0.810553
2 - Training accuracy: 0.817119 / Test accuracy : 0.814637
3 - Training accuracy: 0.817588 / Test accuracy : 0.813516
4 - Training accuracy: 0.816650 / Test accuracy : 0.823845
5 - Training accuracy: 0.816753 / Test accuracy : 0.820802
6 - Training accuracy: 0.816879 / Test accuracy : 0.815197
7 - Training accuracy: 0.816856 / Test accuracy : 0.818160

Average test accuracy: 0.8167287212747217
Variance test accuracy: 0.0
Min test accuracy: 0.8167287212747217
Max test accuracy: 0.8167287212747217


lambda_ = 1
gamma = 0.5 

0 - Training accuracy: 0.817062 / Test accuracy : 0.816959
1 - Training accuracy: 0.817577 / Test accuracy : 0.811034
2 - Training accuracy: 0.817188 / Test accuracy : 0.814557
3 - Training accuracy: 0.817542 / Test accuracy : 0.813516
4 - Training accuracy: 0.816673 / Test accuracy : 0.823365
5 - Train

In [49]:
if test == True:
    if method == least_squares_GD:
        create_csv_submission(ids_te, y_pred_final, '../least_squares_GD.csv')
        
    elif method == least_squares_SGD:
        create_csv_submission(ids_te, y_pred_final, '../least_squares_SGD.csv')
        
    elif method == least_squares:
        create_csv_submission(ids_te, y_pred_final, '../least_squares.csv')
        
    elif method == ridge_regression:
        create_csv_submission(ids_te, y_pred_final, '../ridge_regression.csv')
        
    elif method == logistic_regression:
        create_csv_submission(ids_te, y_pred_final, '../logistic_regression.csv')
        
    elif method == reg_logistic_regression:
        create_csv_submission(ids_te, y_pred_final, '../reg_logistic_regression.csv')
        